# **Combine training data with satellite data**

In [ ]:
pip install rioxarray pystac_client planetary_computer

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd
from shapely.geometry import Point

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

In [ ]:
# Load the training data from csv file and display the first few rows to inspect the data
ground_df = pd.read_csv("/kaggle/input/uncorrupted/Training_data_uhi_index_2025-02-18.csv")
ground_df.head()

In [ ]:
ground_df.shape

In [ ]:
# Reads and plots four bands (B04, B08, B06, B01) from the GeoTIFF file.

# Open the GeoTIFF file
tiff_path = "/kaggle/input/s2-cloud-cover-30/S2 (30).tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(tiff_path) as src1:
    band1 = src1.read(1)  # Band [B01]
    band2 = src1.read(2)  # Band [B02]
    band3 = src1.read(3)  # Band [B03]
    band4 = src1.read(4)  # Band [B04]

# Plot the bands in a 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Flatten the axes for easier indexing
axes = axes.flatten()

# Plot the first band (B01)
im1 = axes[0].imshow(band1, cmap='viridis')
axes[0].set_title('Band [B01]')
fig.colorbar(im1, ax=axes[0])


# Plot the second band (B02)
im2 = axes[1].imshow(band2, cmap='viridis')
axes[1].set_title('Band [B02]')
fig.colorbar(im2, ax=axes[1])

# Plot the third band (B03)
im3 = axes[2].imshow(band3, cmap='viridis')                 
axes[2].set_title('Band [B03]')
fig.colorbar(im3, ax=axes[2])

# Plot the fourth band (B04)
im4 = axes[3].imshow(band4, cmap='viridis')
axes[3].set_title('Band [B04]')
fig.colorbar(im4, ax=axes[3])

plt.tight_layout()
plt.show()

In [ ]:
def map_satellite_data_S2(tiff_path, csv_path, buffer_distance=40):
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs
    
    # Read the CSV file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # Define a projected CRS for New York City (UTM Zone 18N)
    projected_crs = CRS.from_epsg(32618)  # EPSG:32618 is UTM Zone 18N
    
    # Convert lat/long to the GeoTIFF's CRS
    transformer = Transformer.from_crs("EPSG:4326", tiff_crs, always_xy=True)
    
    # Lists to store the band values
    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []
    
    # Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
        try:
            # Convert lat, lon to the target CRS (note x=lon, y=lat)
            x, y = transformer.transform(lon, lat)
            
            # Create a GeoDataFrame with the point in the raster's CRS
            gdf = gpd.GeoDataFrame(geometry=[Point(x, y)], crs=tiff_crs)

            # Reproject to a projected CRS (meters) before buffering
            gdf = gdf.to_crs(projected_crs)

            # Apply the buffer in meters
            gdf["geometry"] = gdf.geometry.buffer(buffer_distance)

            # Convert back to the raster's CRS
            gdf = gdf.to_crs(tiff_crs)

            # Clip the raster using the buffered geometry
            masked_data = data.rio.clip(gdf.geometry, all_touched=True, drop=True)
            
            # Calculate the mean value for each band within the buffer (ignoring NaN values)
            B01_values.append(np.nanmean(masked_data.sel(band=1).values))
            B02_values.append(np.nanmean(masked_data.sel(band=2).values))
            B03_values.append(np.nanmean(masked_data.sel(band=3).values))
            B04_values.append(np.nanmean(masked_data.sel(band=4).values))
            B05_values.append(np.nanmean(masked_data.sel(band=5).values))
            B06_values.append(np.nanmean(masked_data.sel(band=6).values))
            B07_values.append(np.nanmean(masked_data.sel(band=7).values))
            B08_values.append(np.nanmean(masked_data.sel(band=8).values))
            B8A_values.append(np.nanmean(masked_data.sel(band=9).values))
            B11_values.append(np.nanmean(masked_data.sel(band=10).values))
            B12_values.append(np.nanmean(masked_data.sel(band=11).values))
            
        except Exception as e:
            # If there's an error (e.g., point outside raster bounds), append NaN
            print(f"Error processing point ({lat}, {lon}): {e}")
            B01_values.append(np.nan)
            B02_values.append(np.nan)
            B03_values.append(np.nan)
            B04_values.append(np.nan)
            B05_values.append(np.nan)
            B06_values.append(np.nan)
            B07_values.append(np.nan)
            B08_values.append(np.nan)
            B8A_values.append(np.nan)
            B11_values.append(np.nan)
            B12_values.append(np.nan)
    
    # Create a DataFrame to store the band values
    result_df = pd.DataFrame()
    result_df['B01'] = B01_values
    result_df['B02'] = B02_values
    result_df['B03'] = B03_values
    result_df['B04'] = B04_values
    result_df['B05'] = B05_values
    result_df['B06'] = B06_values
    result_df['B07'] = B07_values
    result_df['B08'] = B08_values
    result_df['B8A'] = B8A_values
    result_df['B11'] = B11_values
    result_df['B12'] = B12_values
    
    return result_df

In [ ]:
# Mapping satellite data with training data.
S2_data = map_satellite_data_S2('/kaggle/input/s2-cloud-cover-30/S2 (30).tiff', '/kaggle/input/uncorrupted/Training_data_uhi_index_2025-02-18.csv')

In [ ]:
S2_data.describe()

In [ ]:
S2_data.head()

In [ ]:
S2_data

In [ ]:
# Calculate NDVI
S2_data['NDVI'] = (S2_data['B08'] - S2_data['B04']) / (S2_data['B08'] + S2_data['B04'])
S2_data['NDVI'] = S2_data['NDVI'].replace([np.inf, -np.inf], np.nan)

# Calculate NDBI (Normalized Difference Built-Up Index)
# NDBI = (SWIR - NIR) / (SWIR + NIR)
S2_data['NDBI'] = (S2_data['B11'] - S2_data['B08']) / (S2_data['B11'] + S2_data['B08'])
S2_data['NDBI'] = S2_data['NDBI'].replace([np.inf, -np.inf], np.nan)

# Calculate NDWI (Normalized Difference Water Index)
# NDWI = (Green - NIR) / (Green + NIR)
S2_data['NDWI'] = (S2_data['B03'] - S2_data['B08']) / (S2_data['B03'] + S2_data['B08'])
S2_data['NDWI'] = S2_data['NDWI'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [ ]:
# Combining ground data and final data into a single dataset.
uhi_data = combine_two_datasets(ground_df,S2_data)
uhi_data.head()

In [ ]:
uhi_data

In [ ]:
uhi_data.shape

Extract LST from Landsat

In [ ]:
def map_satellite_data_LST(tiff_path, csv_path, buffer_distance=40):
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs
    
    # Read the CSV file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values
    
    # Convert lat/long to the GeoTIFF's CRS
    transformer = Transformer.from_crs("EPSG:4326", tiff_crs, always_xy=True)
    
    # List to store the LST values
    LST_values = []

    # Define a projected CRS for New York City (UTM Zone 18N)
    projected_crs = CRS.from_epsg(32618)  # EPSG:32618 is UTM Zone 18N
    
    # Iterate over the latitudes and longitudes, and extract the corresponding LST values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
        try:
            # Convert lat, lon to the target CRS (note x=lon, y=lat)
            x, y = transformer.transform(lon, lat)
            
            # Create a GeoDataFrame with the point in the raster's CRS
            gdf = gpd.GeoDataFrame(geometry=[Point(x, y)], crs=tiff_crs)

            # Reproject to a projected CRS (meters) before buffering
            gdf = gdf.to_crs(projected_crs)

            # Apply the buffer in meters
            gdf["geometry"] = gdf.geometry.buffer(buffer_distance)

            # Convert back to the raster's CRS
            gdf = gdf.to_crs(tiff_crs)

            # Clip the raster using the buffered geometry
            masked_data = data.rio.clip(gdf.geometry, all_touched=True, drop=True)
            
            # Calculate the mean LST value within the buffer (ignoring NaN values)
            LST_values.append(np.nanmean(masked_data.sel(band=1).values))
            
        except Exception as e:
            # If there's an error (e.g., point outside raster bounds), append NaN
            print(f"Error processing point ({lat}, {lon}): {e}")
            LST_values.append(np.nan)
    
    # Create a DataFrame to store the LST values
    result_df = pd.DataFrame()
    result_df['LST'] = LST_values
    
    return result_df

In [ ]:
# Mapping satellite data with training data.
LST_data = map_satellite_data_LST('/kaggle/input/train-data/Landsat_LST.tiff', '/kaggle/input/uncorrupted/Training_data_uhi_index_2025-02-18.csv')

In [ ]:
LST_data

In [ ]:
final_data = combine_two_datasets(uhi_data,LST_data)

In [ ]:
final_data.head()

In [ ]:
final_data.shape

In [ ]:
final_data

In [ ]:
output_csv_file = 'training_data_with_satellite.csv'

# Save the DataFrame to CSV
final_data.to_csv(output_csv_file, index=False)

print(f"Data has been successfully saved to {output_csv_file}.")